In [1]:
import scraping_packers_homepage.env as env
llm = env.llm
target_column_list = env.TARGET_COLUMN_LIST
print(target_column_list)

/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.vertexai.VertexAI` was deprecated in langchain-community 0.0.12 and will be removed in 0.2.0. An updated version of the class exists in the langchain-google-vertexai package and should be used instead. To use it run `pip install -U langchain-google-vertexai` and import as `from langchain_google_vertexai import VertexAI`.
  warn_deprecated(


['homepage', 'extract_url', 'company_name', 'mail', 'phone', 'address', 'fax', 'products', 'error']


In [ ]:
import pandas as pd
import scraping_packers_homepage.web_scraper as web_scraper
import importlib
# 모듈이 수정된 후에, 다음 코드를 실행하여 모듈을 다시 로드합니다.
importlib.reload(web_scraper)


url_list = [
    #'https://www.naturzgroup.com/',  # 60
    #'https://kayalfoods.in/',  # 4556
    "http://www.great-lines.com/"
    #'https://www.torresyribelles.com/home2'
]

result_df = pd.DataFrame(columns=target_column_list, dtype=str)
for url in url_list:
    sub_df = web_scraper.run(url, llm, target_column_list, required_column_list=["mail", "phone", "products"])
    result_df = result_df.append(sub_df, ignore_index=True)

len(result_df)

In [ ]:
def unique_list(series):
    # NaN 값을 제거하고, 중복을 제거한 후 리스트로 변환
    return series.dropna().drop_duplicates().tolist()


# 'homepage'를 기준으로 그룹화하고 각 컬럼에 unique_list 함수 적용
grouped_df = result_df.groupby('homepage').agg(unique_list).reset_index()

def convert_to_string(value):
    # 값이 리스트인 경우
    if isinstance(value, list):
        # 빈 리스트인 경우 None 반환
        if not value:
            return ''
        # 리스트의 요소들을 문자열로 결합
        return ";\n".join(str(item) for item in value if pd.notna(item) and isinstance(item, str) and item)
    # 리스트가 아닌 경우, 값을 그대로 반환
    return value

# 'homepage' 컬럼을 제외한 모든 컬럼에 대해 함수 적용
for col in grouped_df.columns:
    if col != 'homepage':
        grouped_df[col] = grouped_df[col].apply(convert_to_string)

print(grouped_df)


In [ ]:
print(grouped_df["products"][0])

In [ ]:
import csv
grouped_df.to_csv('./output/check.csv', quoting=csv.QUOTE_ALL, index=False)

# 작업 방법 노트!!!!

1. 컬럼은 "이메일 작업 대상" 이 있는곳까지만 있는지 체크.
2. 작업대상 컬럼들은 진짜 다음 스텝에서 작업해야하는 경우에만 체크해야한다.
3. 홈페이지 크롤링 통해서 추가 데이터가 수집된 경우 작업대상을 그때그때 마킹한다.
4. 작업 마무리 되면 gcs/.../raw  폴더에 drop

## 고민 포인트
> => homepage 크롤링의 작업 소요 시간이 길다. -> 분산처리 필요.
>    => 분산처리를 어떻게 구성할 것인가?
> => 한번 작업한 homepage 에 대한 상세 내역을 따로 저장해둘것인가?
>    => 작업한 이력이 확인되면 저장된 값을 그대로 갖다 쓰는 방향으로?

## 고민 끝나고 작업하기로! 일단 ai 챗봇하러 간당... 20240130

In [36]:
import pandas as pd
new_df = pd.read_csv('/Users/yoonhae/Downloads/크롤링 데이터 정리_cotton - GMP+ FRA,FSA 1차정리 (2).csv')

job_config = {
    "homepage_column": '홈페이지 주소',
    "target_column_dict": {
        "address": {
            "read_col_list": ['국가(영어로)', '주소 raw'],
            "write_col": '주소 raw',
        },
        "products": {
            "read_col_list": ['SKU raw', '취급 SKU1'],
            "write_col": 'SKU raw',
        },
        "phone": {
            "read_col_list": ['전화번호', '기존 전화번호'],
            "write_col": '기존 전화번호',
        },
        "mail": {
            "read_col_list": ['대표 이메일'],
            "write_col": '대표 이메일',
        }
    }
}



def unique_list(series):
    # NaN 값을 제거하고, 중복을 제거한 후 리스트로 변환
    return series.dropna().drop_duplicates().tolist()


def convert_to_string(value):
    # 값이 리스트인 경우
    if isinstance(value, list):
        # 빈 리스트인 경우 None 반환
        if not value:
            return ''
        # 리스트의 요소들을 문자열로 결합
        return ";\n".join(str(item) for item in value if pd.notna(item) and isinstance(item, str) and item)
    # 리스트가 아닌 경우, 값을 그대로 반환
    return value

In [42]:

import scraping_packers_homepage.web_scraper as web_scraper
import importlib
# 모듈이 수정된 후에, 다음 코드를 실행하여 모듈을 다시 로드합니다.
importlib.reload(web_scraper)


homepage_col = job_config['homepage_column']
target_column_dict = job_config['target_column_dict']
new_df = new_df
for index, row in new_df.iterrows():
    # 홈페이지 주소가 있는 경우
    if not pd.isna(row[homepage_col]):
        required_column_list = []
        # 수집할 컬럼리스트 추출
        for check_col, job_info in target_column_dict.items():
            if any(not pd.isna(row[col]) for col in job_info['read_col_list'] if col in row):
                # 이미 있어서 구지 수집할 필요없음.
                pass
            else:
                required_column_list.append(check_col)
        
        print(row.tolist())
        print(row[homepage_col], required_column_list)

        # 필요한 데이터가 있는 경우
        if required_column_list:
            # 스크래핑 작업 진행.
            sub_df = web_scraper.run(row[homepage_col], llm, target_column_list, required_column_list=required_column_list)
            # 사이트별 하나로 그루핑
            sub_df = sub_df.groupby('homepage').agg(unique_list).reset_index()

            for col in required_column_list:
                # 값이 있는 경우
                if sub_df[col].notna().any() and not sub_df[col].empty:
                    # 그루핑 값을 string 기반으로 변경
                    sub_df[col] = sub_df[col].apply(convert_to_string)
                    # 저장
                    new_df.at[index, target_column_dict[col]['write_col']] = sub_df[col].iloc[0]
                  

['BANKOM d.o.o.', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'Serbia', 'www.bankom.rs', nan, nan, 'ana.jaksic@bankom.rs', nan, nan, '+381 11 319-0052', 'soja;\nsojini proizvodi;\nhrana za ljude;\nhrana za životinje;\nsoje;\nstočne hrane;\npremiksa;\nsirovina za stočnu hranu;\nzamena za mleko za ishranu životinja;\nbaza za sladoled;\nzamena za mleko za ishranu ljudi;\nmesa;\nsuhomesnatih proizvoda', 'Bulevar Nikole Tesle 30a  Belgrade  11080  Serbia', '작업대상', nan, '작업대상', 'ana.jaksic@bankom.rs', nan, 'Serbia', nan, nan, nan, nan, nan, nan, nan, nan]
www.bankom.rs []
['ABA Adriano Barbosa de Azevedo Lda', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'Portugal', 'www.aba.pt', nan, nan, 'helenarodrigues@aba.pt', nan, nan, '+351 239 913 537', 'Transportes de Pulvurlentos;\nLogistica de transporte;\nCisternas de Pulverulentos;\nTransporte geral de carga', 'Rua das Portelas No. 41-47  BARCOUÇO  3

/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.al-ghurair.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.al-ghurair.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  ** [llm] contact 정보 & products 추출
  ** a tag 추출
    [empty]
['ALPHA OSTA LTD - Storage Site.', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'http://www.alpha-osta.lv', nan, nan, 'alpha-osta@alpha-osta.lv', nan, nan, '+371 67351520', '', 'Birztalu street 34  Riga  LV-1015  Latvia', '작업대상', nan, '작업대상', 'alpha-osta@alpha-osta.lv', nan, 'Latvia', nan, nan, nan, nan, nan, nan, nan, nan]
http://www.alpha-osta.lv []
['BRIGHTFORD LTD', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'https://www.brightford.pro/', nan, nan, 'office@brightford.pro', nan, nan, '+380965930542', 'Подсолнечные жирные кислоты;\nПодсолнечный фосфатидный концентрат;\nНерафинированное холодного прессования первого отжима высшего сорта "ЖИВИЛЬНА КРАПЛИНА" (ЖИВИТЕЛЬНАЯ КАПЛЯ);\nНерафинированное вымороженное прессовое масло сорт первый "ЗАПАШНА" (АРОМАТНОЕ);\nНаливные грузы;\nФасованная продукция;\nГорчица;\nМайонез;\n

/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.brenntag.cz'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.brenntag.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


    [404] : 조회 실패
['AGROSPOL Czech, spol. s.r.o.', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'https://www.agrospol.cz', nan, nan, 'silhanek@agrospol.cz', nan, nan, '+420 371 522 745', '', 'Zhůř 133  Nepomuk  335 01  Czech Republic', '작업대상', nan, '작업대상', 'silhanek@agrospol.cz', nan, 'Czech', nan, nan, nan, nan, nan, nan, nan, nan]
https://www.agrospol.cz []
['Czekopol s.r.o.', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'www.czekopol.cz', nan, nan, 'marek.trzaskalik@czekopol.cz', nan, nan, '+420 774 157 930', 'zemědělskými komoditami;\nprůmyslovými hnojivy;\nprostředky na ochranu rostlin;\nkrmnými komponenty;\nosiv', 'Na Požáru 593/3  LETOVICE  679 61  Czech Republic', '작업대상', nan, '작업대상', 'marek.trzaskalik@czekopol.cz', nan, 'Czech', nan, nan, nan, nan, nan, nan, nan, nan]
www.czekopol.cz []
['AGRO Teplice, a.s. – warehouse Žatec', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan

...

  ** [llm] contact 정보 & products 추출
    ** split text - chunk : 4
    {
        "contact": {
            "email": null,
            "phone": null,
            "address": null
        },
        "products": [
            "veevoer",
            "advies",
            "Total Feed-aanpak",
            "melkveevoeding",
            "droogstand en opstart",
            "jongvee",
            "ruwvoer",
            "TMR",
            "robotmelken",
            "weidegang",
            "aanpak en producten vleesvee",
            "aanpak en producten legpluimvee",
            "aanpak en producten vleespluimvee",
            "aanpak en producten vermeerdering",
            "aanpak en producten biggen",
            "aanpak en producten zeugen",
            "aanpak en producten vleesvarkens",
            "aanpak en producten HomeMiXX",
            "aanpak en producten ruwvoer",
            "aanpak en producten geitenvoeding",
            "aanpak en producten droogstand en opstart",
            "aan

/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.contitank.eu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  ** [llm] contact 정보 & products 추출
    ** split text - chunk : 2
    {
        "contact": {
            "email": null,
            "phone": null,
            "address": null
        },
        "products": []
    }
    {
        "contact": {
            "email": "info@contitank.eu",
            "phone": "+31 (0) 59 66 325 99",
            "address": "Melasseweg 1, 9936 CX, Farmsum"
        },
        "products": [
            "plantaardige- en dierlijke vetten",
            "oli\u00ebn"
        ]
    }
['Chemisafe A/S', nan, nan, nan, '(인증서) GMP+ FSA', nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 'www.chemisafe.dk', nan, nan, 'ma@chemisafe.dk', nan, nan, nan, nan, 'Erhvervsparken 15  GRENAA  DK-8500  Denmark', '작업대상', nan, '작업대상', 'ma@chemisafe.dk', nan, 'Denmark', nan, nan, nan, nan, nan, nan, nan, nan]
www.chemisafe.dk ['products', 'phone']
** [main page] 조회 - www.chemisafe.dk


/Users/yoonhae/.pyenv/versions/3.8.12/envs/crawling/lib/python3.8/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.chemisafe.dk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


  ** [llm] contact 정보 & products 추출
    {
        "contact": {
            "email": null,
            "phone": null,
            "address": null
        },
        "products": [
            "vintersalt",
            "aerosoler",
            "bek\u00e6mpelsesmidler"
        ]
    }
  ** a tag 추출
    [empty]


In [121]:
new_df = pd.read_csv('output/크롤링 데이터 정리_cotton - GMP+ FRA,FSA 1차정리_from_homepage_20240129.csv')


In [123]:
if "new_contry" in new_df.columns:
    new_df["국가(영어로)"] = new_df["new_contry"]

In [126]:
import csv
filtered_df = new_df.iloc[:, :new_df.columns.get_loc('이메일 작업 대상') + 1]
filtered_df.to_csv("./output/크롤링 데이터 정리_cotton - GMP+ FRA,FSA 1차정리_from_homepage_20240130.csv", quoting=csv.QUOTE_ALL, index=False)

In [124]:

import numpy as np    

new_df["주소 작업대상"] = ''
new_df['SKU 작업대상'] = ''
new_df['이메일 작업 대상'] = ''

new_df['주소 작업대상'] = np.where((new_df['주소 raw'].notnull() & new_df['주소 raw'].notna() & (new_df['주소 raw'] != '')) &
                   (new_df['국가(영어로)'].isnull() | new_df['국가(영어로)'].isna() | (new_df['국가(영어로)'] == '')),
                   '작업대상', '')

new_df['SKU 작업대상'] = np.where((new_df['SKU raw'].notnull() & new_df['SKU raw'].notna() & (new_df['SKU raw'] != '')) &
                   (new_df['취급 SKU1'].isnull() | new_df['취급 SKU1'].isna() | (new_df['취급 SKU1'] == '')),
                   '작업대상', '')
if (new_df['참조 이메일(여러개 입력시 콤마(,)로 구분)'].notna() & (new_df['참조 이메일(여러개 입력시 콤마(,)로 구분)'] != '')).any() and \
   ((new_df['대표 이메일'].str.count('@') > 1) | 
    (new_df['대표 이메일'].str.contains('[ ,;]'))).any():
    new_df['이메일 작업 대상'] = np.where((new_df['대표 이메일'].notnull() & new_df['대표 이메일'].notna() & (new_df['대표 이메일'] != '')),
                                '작업대상', '')
    
filtered_df = new_df.iloc[:, :new_df.columns.get_loc('이메일 작업 대상') + 1]
filtered_df.to_csv("./output/크롤링 데이터 정리_cotton - GMP+ FRA,FSA 1차정리_from_homepage_20240130.csv", quoting=csv.QUOTE_ALL, index=False)

,패커명*,패커명2,연락단계*,검증단계*,획득경로1,획득경로2,획득경로3,획득경로4,획득경로5,취급 SKU1,...,전화번호,대표 이메일,"참조 이메일(여러개 입력시 콤마(,)로 구분)",중복여부 표시,기존 전화번호,SKU raw,주소 raw,주소 작업대상,SKU 작업대상,이메일 작업 대상
0,CBS Bio Platforms Inc.,NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,rpatterson@canadianbio.com,NaN,NaN,905-432-3558,NaN,Thornton Rd S 1345 OSHAWA L1J 8C4 Canada,,,작업대상
1,ADM Agtri-Industries Company,NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,519 9724502,NaN,"Maplewood Drive 5550 WINDSOR, ONTARIO N9C 0B...",,,
2,"Biomin Vietnam Co., Ltd.",NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,tranthi.hong@biomin.net,NaN,NaN,(024) 38765632,NaN,"Duong Danh Hamlet, Duong Xa Commune, Gia Lam D...",,,작업대상
3,Angler BioChemLab PT,NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,QA@anglerlab.co.id,NaN,NaN,0317456111,NaN,"Jalan Sawo 17-19 Bringin, Sambikerep - Surab...",,,작업대상
4,Baja Agro Internacional SA de CV,NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,agroin@yucca.com.mx,NaN,NaN,+525 646177 - 0475,NaN,Km. 95 Carretera Ensenada - San Felipe Deleg....,,,작업대상
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4379,Dombeek & van Bakkum,NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0031,NaN,Meeuwenstraat 12B ROTTERDAM 3071 PE The Net...,,,
4380,Barilla France SAS,NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0327147700,NaN,Parc d'activites de la Vallee de l'Escaut ONN...,,,
4381,BSR van Uden Stevedoring B.V.,NaN,NaN,NaN,(인증서) GMP+ FSA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0031,NaN,Gustoweg 68 Rotterdam 3029 AS The Netherlands,,,
4382,BayWa AG Freiberg/Neckar,NaN,NaN,NaN,(인증서) GMP+ FRA,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0049,NaN,Steinheimer Straße 9 FREIBERG AM NECKAR 7169...,,,
